In [29]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from sklearn.compose import make_column_transformer 
from sklearn.compose import make_column_selector
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, accuracy_score, r2_score
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold, GridSearchCV, KFold
from sklearn.pipeline import Pipeline
import os 
import warnings
warnings.filterwarnings('ignore')

In [3]:
boston = pd.read_csv('Boston.csv')
boston

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273,21.0,393.45,6.48,22.0


In [24]:
X = boston.drop(columns='medv')
y = boston.medv

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=24,test_size=0.3)


In [25]:
ohe = OneHotEncoder(sparse_output=False, drop='first').set_output(transform='pandas')
scaler_mm = MinMaxScaler()
scaler_std = StandardScaler()
ct = make_column_transformer(('passthrough', make_column_selector(dtype_exclude=object)  ),
                             (ohe, make_column_selector(dtype_include=object) ),
                            verbose_feature_names_out=False).set_output(transform='pandas')
knnr = KNeighborsRegressor()
pipe = Pipeline([('CT',ct),('SCL',None),('KNNR',knnr)])
pipe.get_params()

{'memory': None,
 'steps': [('CT',
   ColumnTransformer(transformers=[('passthrough', 'passthrough',
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x000001CB71EB0950>),
                                   ('onehotencoder',
                                    OneHotEncoder(drop='first',
                                                  sparse_output=False),
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x000001CB71E913D0>)],
                     verbose_feature_names_out=False)),
  ('SCL', None),
  ('KNNR', KNeighborsRegressor())],
 'verbose': False,
 'CT': ColumnTransformer(transformers=[('passthrough', 'passthrough',
                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001CB71EB0950>),
                                 ('onehotencoder',
                                  OneHotEncoder(drop='first',
                      

In [31]:
kfold = KFold(n_splits=5, random_state=24, 
                        shuffle=True)
# here we are checking which KNN value is best in the range
params = {'KNNR__n_neighbors': np.arange(1,11),
         'KNNR__metric':['cityblock','minkowski','manhatan','haversine'],
          'SCL':[scaler_mm, scaler_std, None]}
# Setting the parameter ng_log_loss in scoring
gcv = GridSearchCV(pipe, param_grid=params,
                   scoring='r2',          
                   cv=kfold)
gcv.fit(X,y)
print("Best Parameters: ",gcv.best_params_)
print("Best Score: ",gcv.best_score_)

pd_cv = pd.DataFrame(gcv.cv_results_)
print("Shape: ",pd_cv.shape)

Best Parameters:  {'KNNR__metric': 'cityblock', 'KNNR__n_neighbors': np.int64(3), 'SCL': StandardScaler()}
Best Score:  0.7875954797972841
Shape:  (120, 16)
